<a href="https://colab.research.google.com/github/anablima/MBA_DataEngineer/blob/main/Bootcamp_Data_Engineering_Exercicios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Temos duas bases de dados pessoais (fictícios)


*   cadastro.csv (dados de cadastro)
*   dados_medicos.csv (registros médicos)

Temos que unificar as duas bases e criar uma nova base agrupada

Código localizado em https://github.com/renzoziegler/data_engineering_bootcamp


Exercício 1:


*   Importar o Pandas
*   Instalar e importar a biblioteca fuzzywuzzy (será importante mais pra frente)
*   Ler os arquivos CSV fornecidos
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/cadastro.csv
** https://raw.githubusercontent.com/renzoziegler/data_engineering_bootcamp/main/dados_medicos.csv
*   Realizar uma análise exploratória dos dados



In [ ]:
#Seu código aqui

Para análise exploratória, vamos entender as bases de dados: seu tamanho, quais dados estão disponíveis, qual o formato deles, se estão completos ou não.....

In [ ]:
#Seu código aqui

Exercício 2: Pré-processamento
*   Quais tarefas vocês listam como necessárias?
 *  Padronização de campos
 *  Substituir valores
 *  Retirar caracteres especiais
*   Listar e executar!

In [ ]:
#Seu código aqui

Exercício 3: Indexação
*    Listar possíveis blocking Keys
*    Definir uma a ser utilizada
*    Criar os blocos
*    Listar os possíveis matches em cada bloco

In [ ]:
#Seu código aqui

Exercício 4: Comparação par a par
*    Desenvolver um algoritmo de comparação de cada match potencial do exercício anterior
*    Definir uma nota de similaridade para cada comparação
 *   Para cada parâmetro

In [ ]:
#Seu código aqui

Exercício 5: Classificação
*    Classificar cada par, baseado num threshold
*    Definir um peso para cada parâmetro

In [ ]:
#Seu código aqui

Exercício 6: Avaliação
*     Buscar uma amostra das classificações (de 10 a 20 registros)
*     Classificar cada uma em TP, TN, FP, FN
*     Calcular acurácia, precisão e recall

In [ ]:
#Seu código aqui